In [1]:
import tensorflow as tf
config = tf.ConfigProto(allow_soft_placement=True) 
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

D:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dty

In [2]:
import colorsys
import os
from timeit import default_timer as timer

import numpy as np
from keras import backend as K
from keras.models import load_model
from keras.layers import Input
from PIL import Image, ImageFont, ImageDraw

from yolo3.model import yolo_eval, yolo_body, tiny_yolo_body
from yolo3.utils import letterbox_image
import os
from keras.utils import multi_gpu_model

Using TensorFlow backend.


In [3]:
class YOLO(object):
    _defaults = {
        "model_path": 'model_data/trained_weights_final.h5', #指定YOLO訓練完成權重檔路徑及名稱
        "anchors_path": 'model_data/yolo_anchors.txt', #指定錨點定義檔路徑及名稱
        "classes_path": 'model_data/my_classes.txt', #指定自定義標籤檔路徑及名稱
        "score" : 0.5, #最低置信度門檻(0.01~0.99)
        "iou" : 0.5, #重疊區比例(0.01~1.0)
        "model_image_size" : (416, 416), #影像尺寸
        "gpu_num" : 1, #使用GPU數量
    }

    @classmethod
    def get_defaults(cls, n):
        if n in cls._defaults:
            return cls._defaults[n]
        else:
            return "Unrecognized attribute name '" + n + "'"

    def __init__(self, **kwargs):
        self.__dict__.update(self._defaults) # set up default values
        self.__dict__.update(kwargs) # and update with user overrides
        self.class_names = self._get_class()
        self.anchors = self._get_anchors()
        self.sess = K.get_session()
        self.boxes, self.scores, self.classes = self.generate()

    def _get_class(self):
        classes_path = os.path.expanduser(self.classes_path)
        with open(classes_path) as f:
            class_names = f.readlines()
        class_names = [c.strip() for c in class_names]
        return class_names

    def _get_anchors(self):
        anchors_path = os.path.expanduser(self.anchors_path)
        with open(anchors_path) as f:
            anchors = f.readline()
        anchors = [float(x) for x in anchors.split(',')]
        return np.array(anchors).reshape(-1, 2)

    def generate(self):
        model_path = os.path.expanduser(self.model_path)
        assert model_path.endswith('.h5'), 'Keras model or weights must be a .h5 file.'

        # Load model, or construct model and load weights.
        num_anchors = len(self.anchors)
        num_classes = len(self.class_names)
        is_tiny_version = num_anchors==6 # default setting
        try:
            self.yolo_model = load_model(model_path, compile=False)
        except:
            self.yolo_model = tiny_yolo_body(Input(shape=(None,None,3)), num_anchors//2, num_classes) \
                if is_tiny_version else yolo_body(Input(shape=(None,None,3)), num_anchors//3, num_classes)
            self.yolo_model.load_weights(self.model_path) # make sure model, anchors and classes match
        else:
            assert self.yolo_model.layers[-1].output_shape[-1] == \
                num_anchors/len(self.yolo_model.output) * (num_classes + 5), \
                'Mismatch between model and given anchor and class sizes'

        print('{} model, anchors, and classes loaded.'.format(model_path))

        # Generate colors for drawing bounding boxes.
        hsv_tuples = [(x / len(self.class_names), 1., 1.)
                      for x in range(len(self.class_names))]
        self.colors = list(map(lambda x: colorsys.hsv_to_rgb(*x), hsv_tuples))
        self.colors = list(
            map(lambda x: (int(x[0] * 255), int(x[1] * 255), int(x[2] * 255)),
                self.colors))
        np.random.seed(10101)  # Fixed seed for consistent colors across runs.
        np.random.shuffle(self.colors)  # Shuffle colors to decorrelate adjacent classes.
        np.random.seed(None)  # Reset seed to default.

        # Generate output tensor targets for filtered bounding boxes.
        self.input_image_shape = K.placeholder(shape=(2, ))
        if self.gpu_num>=2:
            self.yolo_model = multi_gpu_model(self.yolo_model, gpus=self.gpu_num)
        boxes, scores, classes = yolo_eval(self.yolo_model.output, self.anchors,
                len(self.class_names), self.input_image_shape,
                score_threshold=self.score, iou_threshold=self.iou)
        return boxes, scores, classes

    def detect_image(self, image):
        start = timer()

        if self.model_image_size != (None, None):
            assert self.model_image_size[0]%32 == 0, 'Multiples of 32 required'
            assert self.model_image_size[1]%32 == 0, 'Multiples of 32 required'
            boxed_image = letterbox_image(image, tuple(reversed(self.model_image_size)))
        else:
            new_image_size = (image.width - (image.width % 32),
                              image.height - (image.height % 32))
            boxed_image = letterbox_image(image, new_image_size)
        image_data = np.array(boxed_image, dtype='float32')

        print(image_data.shape)
        image_data /= 255.
        image_data = np.expand_dims(image_data, 0)  # Add batch dimension.

        out_boxes, out_scores, out_classes = self.sess.run(
            [self.boxes, self.scores, self.classes],
            feed_dict={
                self.yolo_model.input: image_data,
                self.input_image_shape: [image.size[1], image.size[0]],
                K.learning_phase(): 0
            })

        print('Found {} boxes for {}'.format(len(out_boxes), 'img'))

        font = ImageFont.truetype(font='font/FiraMono-Medium.otf',
                    size=np.floor(3e-2 * image.size[1] + 0.5).astype('int32'))
        thickness = (image.size[0] + image.size[1]) // 300

        for i, c in reversed(list(enumerate(out_classes))):
            predicted_class = self.class_names[c]
            box = out_boxes[i]
            score = out_scores[i]

            label = '{} {:.2f}'.format(predicted_class, score)
            draw = ImageDraw.Draw(image)
            label_size = draw.textsize(label, font)

            top, left, bottom, right = box
            top = max(0, np.floor(top + 0.5).astype('int32'))
            left = max(0, np.floor(left + 0.5).astype('int32'))
            bottom = min(image.size[1], np.floor(bottom + 0.5).astype('int32'))
            right = min(image.size[0], np.floor(right + 0.5).astype('int32'))
            print(label, (left, top), (right, bottom))

            if top - label_size[1] >= 0:
                text_origin = np.array([left, top - label_size[1]])
            else:
                text_origin = np.array([left, top + 1])

            # My kingdom for a good redistributable image drawing library.
            for i in range(thickness):
                draw.rectangle(
                    [left + i, top + i, right - i, bottom - i],
                    outline=self.colors[c])
            draw.rectangle(
                [tuple(text_origin), tuple(text_origin + label_size)],
                fill=self.colors[c])
            draw.text(text_origin, label, fill=(0, 0, 0), font=font)
            del draw

        end = timer()
        print(end - start)
        return image

    def close_session(self):
        self.sess.close()

In [4]:
def detect_video(yolo, video_path, output_path=""):
    import cv2
    vid = cv2.VideoCapture(video_path)
    # 使用 XVID 編碼
    fourcc = cv2.VideoWriter_fourcc(*'XVID')

    # 建立 VideoWriter 物件，輸出影片至 output.avi
    # FPS 值為 30.0，解析度為 1280*720
    out = cv2.VideoWriter(output_path+'Kangaroo-detection.avi', fourcc, 30.0, (1280, 720))
    
#     if not vid.isOpened():
#         raise IOError("Couldn't open webcam or video")
        
#     video_FourCC    = int(vid.get(cv2.CAP_PROP_FOURCC))
#     video_fps       = vid.get(cv2.CAP_PROP_FPS)
#     video_size      = (int(vid.get(cv2.CAP_PROP_FRAME_WIDTH)),
#                         int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT)))
     
#     if output_path != "":
#         isOutput = True
#     else:
#         isOutput = False
    
#     if isOutput:
#         print("!!! TYPE:", type(output_path), type(video_FourCC), type(video_fps), type(video_size))
#         out = cv2.VideoWriter(output_path, video_FourCC, video_fps, video_size)
        
    accum_time = 0
    curr_fps = 0
    fps = "FPS: ??"
    prev_time = timer()
    
    while True:
        return_value, frame = vid.read()
        
        if frame is None:
             break
        
        image = Image.fromarray(frame)
        image = yolo.detect_image(image)
        result = np.asarray(image)
        curr_time = timer()
        exec_time = curr_time - prev_time
        prev_time = curr_time
        accum_time = accum_time + exec_time
        curr_fps = curr_fps + 1
        
        if accum_time > 1:
            accum_time = accum_time - 1
            fps = "FPS: " + str(curr_fps)
            curr_fps = 0
            
        cv2.putText(result, text=fps, org=(3, 15), fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale=0.50, color=(255, 0, 0), thickness=2)
        cv2.namedWindow("result", cv2.WINDOW_NORMAL)
        cv2.imshow("result", result)
        out.write(result)
        
#         if isOutput:
#             out.write(result)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
    # 釋放所有資源
    vid.release()
    out.release()
    cv2.destroyAllWindows()
        
    yolo.close_session()

In [5]:
yolo=YOLO()

model_data/trained_weights_final.h5 model, anchors, and classes loaded.


In [6]:
path ='D://Program Code//Python//ML//DL-馬拉松//Final//Raccoon.mp4' #指定待測影像檔案路徑及名稱
Output = 'D://Program Code//Python//ML//DL-馬拉松//Final//' 
detect_video(yolo,path,Output)

(416, 416, 3)
Found 0 boxes for img
2.4051788999999997
(416, 416, 3)
Found 0 boxes for img
0.11515580000000014
(416, 416, 3)
Found 0 boxes for img
0.1139190000000001
(416, 416, 3)
Found 0 boxes for img
0.11198940000000013
(416, 416, 3)
Found 0 boxes for img
0.11193300000000006
(416, 416, 3)
Found 0 boxes for img
0.11267259999999979
(416, 416, 3)
Found 0 boxes for img
0.11362359999999994
(416, 416, 3)
Found 0 boxes for img
0.1119570999999997
(416, 416, 3)
Found 0 boxes for img
0.11235160000000022
(416, 416, 3)
Found 0 boxes for img
0.10987139999999984
(416, 416, 3)
Found 0 boxes for img
0.11048139999999984
(416, 416, 3)
Found 0 boxes for img
0.1136784999999998
(416, 416, 3)
Found 0 boxes for img
0.11253160000000006
(416, 416, 3)
Found 0 boxes for img
0.1109176000000005
(416, 416, 3)
Found 0 boxes for img
0.1112649999999995
(416, 416, 3)
Found 0 boxes for img
0.1101636000000008
(416, 416, 3)
Found 0 boxes for img
0.11080549999999967
(416, 416, 3)
Found 0 boxes for img
0.10982860000000016

Found 0 boxes for img
0.10985149999999777
(416, 416, 3)
Found 0 boxes for img
0.11138329999999996
(416, 416, 3)
Found 0 boxes for img
0.10922610000000077
(416, 416, 3)
Found 0 boxes for img
0.11070660000000032
(416, 416, 3)
Found 0 boxes for img
0.1102761000000001
(416, 416, 3)
Found 0 boxes for img
0.10932599999999937
(416, 416, 3)
Found 0 boxes for img
0.10997970000000024
(416, 416, 3)
Found 0 boxes for img
0.11077239999999833
(416, 416, 3)
Found 0 boxes for img
0.10998520000000056
(416, 416, 3)
Found 0 boxes for img
0.11007370000000094
(416, 416, 3)
Found 0 boxes for img
0.11433920000000342
(416, 416, 3)
Found 0 boxes for img
0.11165859999999839
(416, 416, 3)
Found 0 boxes for img
0.1101756000000016
(416, 416, 3)
Found 0 boxes for img
0.1104744999999987
(416, 416, 3)
Found 0 boxes for img
0.11020689999999789
(416, 416, 3)
Found 0 boxes for img
0.11040900000000065
(416, 416, 3)
Found 0 boxes for img
0.10990269999999924
(416, 416, 3)
Found 0 boxes for img
0.11011899999999741
(416, 416

Found 1 boxes for img
raccoon 0.95 (498, 4) (1207, 571)
0.11264119999999878
(416, 416, 3)
Found 1 boxes for img
raccoon 0.91 (509, 0) (1206, 581)
0.11203379999999896
(416, 416, 3)
Found 1 boxes for img
raccoon 0.89 (518, 0) (1206, 595)
0.11200279999999907
(416, 416, 3)
Found 1 boxes for img
raccoon 0.88 (514, 0) (1214, 589)
0.11238469999999978
(416, 416, 3)
Found 1 boxes for img
raccoon 0.85 (503, 0) (1225, 595)
0.11176290000000222
(416, 416, 3)
Found 1 boxes for img
raccoon 0.88 (503, 0) (1228, 600)
0.11261499999999813
(416, 416, 3)
Found 1 boxes for img
raccoon 0.88 (476, 0) (1280, 558)
0.11203379999999896
(416, 416, 3)
Found 1 boxes for img
raccoon 0.87 (480, 0) (1280, 553)
0.11175579999999741
(416, 416, 3)
Found 1 boxes for img
raccoon 0.87 (478, 0) (1280, 557)
0.11236519999999928
(416, 416, 3)
Found 1 boxes for img
raccoon 0.82 (504, 0) (1227, 587)
0.11231889999999822
(416, 416, 3)
Found 1 boxes for img
raccoon 0.76 (505, 0) (1280, 567)
0.1239143000000027
(416, 416, 3)
Found 1 box

Found 1 boxes for img
raccoon 0.82 (197, 25) (1280, 667)
0.11218139999999721
(416, 416, 3)
Found 1 boxes for img
raccoon 0.74 (194, 26) (1280, 673)
0.12169289999999933
(416, 416, 3)
Found 1 boxes for img
raccoon 0.66 (216, 36) (1280, 664)
0.11166810000000282
(416, 416, 3)
Found 1 boxes for img
raccoon 0.66 (230, 22) (1274, 671)
0.11282659999999822
(416, 416, 3)
Found 1 boxes for img
raccoon 0.78 (231, 27) (1271, 665)
0.11219719999999711
(416, 416, 3)
Found 1 boxes for img
raccoon 0.76 (212, 24) (1280, 672)
0.11198499999999711
(416, 416, 3)
Found 1 boxes for img
raccoon 0.78 (180, 22) (1280, 679)
0.1118025999999972
(416, 416, 3)
Found 1 boxes for img
raccoon 0.77 (172, 31) (1280, 666)
0.11448500000000195
(416, 416, 3)
Found 1 boxes for img
raccoon 0.81 (160, 22) (1280, 673)
0.11201220000000234
(416, 416, 3)
Found 1 boxes for img
raccoon 0.88 (151, 24) (1280, 674)
0.11193560000000247
(416, 416, 3)
Found 1 boxes for img
raccoon 0.86 (160, 29) (1280, 673)
0.11271729999999991
(416, 416, 3)


Found 1 boxes for img
raccoon 0.96 (94, 0) (1280, 713)
0.11420189999999764
(416, 416, 3)
Found 1 boxes for img
raccoon 0.96 (131, 0) (1280, 715)
0.11212550000000476
(416, 416, 3)
Found 1 boxes for img
raccoon 0.96 (106, 0) (1280, 707)
0.1124553000000077
(416, 416, 3)
Found 1 boxes for img
raccoon 0.94 (0, 0) (1280, 720)
0.11283170000000098
(416, 416, 3)
Found 1 boxes for img
raccoon 0.98 (88, 0) (1280, 691)
0.12032189999999332
(416, 416, 3)
Found 1 boxes for img
raccoon 0.97 (108, 0) (1280, 692)
0.11210610000000543
(416, 416, 3)
Found 1 boxes for img
raccoon 0.97 (100, 0) (1280, 686)
0.11192009999999186
(416, 416, 3)
Found 1 boxes for img
raccoon 0.97 (0, 0) (1280, 717)
0.11097409999999286
(416, 416, 3)
Found 1 boxes for img
raccoon 0.98 (47, 0) (1280, 705)
0.11241169999999556
(416, 416, 3)
Found 1 boxes for img
raccoon 0.97 (35, 0) (1280, 707)
0.11226769999998965
(416, 416, 3)
Found 1 boxes for img
raccoon 0.98 (20, 0) (1280, 696)
0.11226440000000082
(416, 416, 3)
Found 1 boxes for im

...

Found 0 boxes for img
0.1103610000000117
(416, 416, 3)
Found 1 boxes for img
raccoon 0.58 (239, 100) (835, 720)
0.11159029999998893
(416, 416, 3)
Found 1 boxes for img
raccoon 0.59 (234, 70) (841, 720)
0.1141034999999988
(416, 416, 3)
Found 0 boxes for img
0.11148180000000707
(416, 416, 3)
Found 0 boxes for img
0.11011590000001092
(416, 416, 3)
Found 0 boxes for img
0.11007609999998635
(416, 416, 3)
Found 0 boxes for img
0.11184959999999933
(416, 416, 3)
Found 1 boxes for img
raccoon 0.53 (229, 167) (840, 596)
0.11247469999997861
(416, 416, 3)
Found 1 boxes for img
raccoon 0.76 (246, 134) (820, 626)
0.1139682000000164
(416, 416, 3)
Found 1 boxes for img
raccoon 0.64 (229, 76) (661, 657)
0.11219270000000847
(416, 416, 3)
Found 1 boxes for img
raccoon 0.64 (224, 76) (653, 657)
0.11185610000001134
(416, 416, 3)
Found 1 boxes for img
raccoon 0.58 (234, 52) (666, 678)
0.11306569999999283
(416, 416, 3)
Found 0 boxes for img
0.12080439999999726
(416, 416, 3)
Found 0 boxes for img
0.1094189000

Found 1 boxes for img
raccoon 0.86 (266, 0) (868, 709)
0.11387159999998175
(416, 416, 3)
Found 1 boxes for img
raccoon 0.86 (260, 8) (871, 703)
0.11461819999999534
(416, 416, 3)
Found 1 boxes for img
raccoon 0.78 (320, 32) (918, 672)
0.11247740000001727
(416, 416, 3)
Found 1 boxes for img
raccoon 0.81 (328, 38) (912, 678)
0.11198399999997832
(416, 416, 3)
Found 1 boxes for img
raccoon 0.86 (307, 43) (930, 669)
0.11245610000000283
(416, 416, 3)
Found 1 boxes for img
raccoon 0.92 (284, 34) (969, 666)
0.11225660000002335
(416, 416, 3)
Found 1 boxes for img
raccoon 0.95 (271, 36) (983, 674)
0.11994190000001481
(416, 416, 3)
Found 1 boxes for img
raccoon 0.96 (263, 15) (1003, 699)
0.11221449999999322
(416, 416, 3)
Found 1 boxes for img
raccoon 0.97 (254, 14) (1016, 705)
0.11178820000000655
(416, 416, 3)
Found 1 boxes for img
raccoon 0.95 (246, 31) (1024, 697)
0.1127341000000115
(416, 416, 3)
Found 1 boxes for img
raccoon 0.93 (243, 42) (1027, 691)
0.11228630000002227
(416, 416, 3)
Found 1 b

Found 1 boxes for img
raccoon 0.98 (0, 0) (1280, 711)
0.11244639999998185
(416, 416, 3)
Found 1 boxes for img
raccoon 0.98 (0, 0) (1280, 685)
0.1124415999999826
(416, 416, 3)
Found 1 boxes for img
raccoon 0.99 (0, 0) (1280, 682)
0.11265140000000429
(416, 416, 3)
Found 1 boxes for img
raccoon 0.99 (0, 0) (1280, 688)
0.11244840000000522
(416, 416, 3)
Found 1 boxes for img
raccoon 0.99 (0, 0) (1280, 692)
0.11178090000001362
(416, 416, 3)
Found 1 boxes for img
raccoon 0.99 (0, 0) (1280, 687)
0.11197119999999927
(416, 416, 3)
Found 1 boxes for img
raccoon 0.99 (0, 0) (1280, 673)
0.11499259999999367
(416, 416, 3)
Found 1 boxes for img
raccoon 0.98 (0, 8) (1280, 660)
0.11227699999997753
(416, 416, 3)
Found 1 boxes for img
raccoon 0.98 (0, 0) (1280, 666)
0.11380349999998884
(416, 416, 3)
Found 1 boxes for img
raccoon 0.97 (1, 13) (1280, 648)
0.11399380000000292
(416, 416, 3)
Found 1 boxes for img
raccoon 0.98 (0, 8) (1280, 652)
0.11218489999998837
(416, 416, 3)
Found 1 boxes for img
raccoon 0.

Found 0 boxes for img
0.11332850000002281
(416, 416, 3)
Found 0 boxes for img
0.11052579999997647
(416, 416, 3)
Found 1 boxes for img
raccoon 0.74 (103, 0) (629, 720)
0.112891999999988
(416, 416, 3)
Found 1 boxes for img
raccoon 0.72 (130, 0) (598, 720)
0.1123185999999805
(416, 416, 3)
Found 1 boxes for img
raccoon 0.63 (137, 0) (586, 720)
0.11271170000000552
(416, 416, 3)
Found 1 boxes for img
raccoon 0.58 (132, 0) (593, 720)
0.11316019999998161
(416, 416, 3)
Found 0 boxes for img
0.11046460000000025
(416, 416, 3)
Found 0 boxes for img
0.10995170000001053
(416, 416, 3)
Found 1 boxes for img
raccoon 0.58 (121, 0) (609, 720)
0.11225769999998647
(416, 416, 3)
Found 1 boxes for img
raccoon 0.51 (126, 0) (613, 720)
0.11252890000000093
(416, 416, 3)
Found 1 boxes for img
raccoon 0.52 (139, 0) (603, 720)
0.11535100000000398
(416, 416, 3)
Found 1 boxes for img
raccoon 0.54 (129, 0) (618, 720)
0.11256979999998862
(416, 416, 3)
Found 1 boxes for img
raccoon 0.51 (125, 0) (619, 720)
0.1142161000

Found 0 boxes for img
0.10995389999999361
(416, 416, 3)
Found 1 boxes for img
kangaroo 0.54 (71, 320) (632, 720)
0.11150420000001304
(416, 416, 3)
Found 1 boxes for img
kangaroo 0.53 (67, 322) (636, 720)
0.11308439999999109
(416, 416, 3)
Found 0 boxes for img
0.11110859999999434
(416, 416, 3)
Found 1 boxes for img
kangaroo 0.55 (68, 315) (629, 720)
0.11195770000000493
(416, 416, 3)
Found 1 boxes for img
kangaroo 0.57 (73, 314) (621, 720)
0.11342460000000187
(416, 416, 3)
Found 1 boxes for img
kangaroo 0.75 (66, 327) (629, 720)
0.1123179999999877
(416, 416, 3)
Found 1 boxes for img
kangaroo 0.76 (61, 330) (631, 720)
0.1140814999999975
(416, 416, 3)
Found 1 boxes for img
kangaroo 0.75 (62, 328) (633, 720)
0.1141960999999867
(416, 416, 3)
Found 1 boxes for img
kangaroo 0.72 (60, 326) (641, 720)
0.11211020000001781
(416, 416, 3)
Found 1 boxes for img
kangaroo 0.73 (52, 326) (650, 720)
0.11226750000000152
(416, 416, 3)
Found 1 boxes for img
kangaroo 0.70 (52, 328) (652, 720)
0.1160515999999

In [ ]:
path ='D://Program Code//Python//ML//DL-馬拉松//Final//Kangaroo.mp4' #指定待測影像檔案路徑及名稱
Output = 'D://Program Code//Python//ML//DL-馬拉松//Final//' 
detect_video(yolo,path,Output)

(416, 416, 3)
Found 1 boxes for img
raccoon 0.51 (464, 0) (1200, 720)
2.4199102000000003
(416, 416, 3)
Found 1 boxes for img
raccoon 0.51 (464, 0) (1200, 720)
0.1232823999999999
(416, 416, 3)
Found 1 boxes for img
raccoon 0.60 (448, 0) (1210, 720)
0.1127391000000002
(416, 416, 3)
Found 1 boxes for img
raccoon 0.51 (63, 0) (447, 720)
0.11410199999999993
(416, 416, 3)
Found 1 boxes for img
raccoon 0.66 (62, 0) (452, 715)
0.11258860000000004
(416, 416, 3)
Found 1 boxes for img
raccoon 0.65 (55, 0) (463, 627)
0.11515159999999991
(416, 416, 3)
Found 1 boxes for img
raccoon 0.73 (56, 0) (461, 631)
0.13728050000000014
(416, 416, 3)
Found 1 boxes for img
raccoon 0.61 (52, 0) (472, 632)
0.11353729999999951
(416, 416, 3)
Found 1 boxes for img
raccoon 0.62 (57, 0) (472, 638)
0.11271710000000024
(416, 416, 3)
Found 0 boxes for img
0.11274599999999957
(416, 416, 3)
Found 0 boxes for img
0.11072940000000031
(416, 416, 3)
Found 1 boxes for img
raccoon 0.53 (94, 0) (1204, 720)
0.11427690000000013
(416

Found 0 boxes for img
0.11237709999999979
(416, 416, 3)
Found 0 boxes for img
0.11879259999999903
(416, 416, 3)
Found 0 boxes for img
0.12095740000000177
(416, 416, 3)
Found 0 boxes for img
0.11037280000000038
(416, 416, 3)
Found 0 boxes for img
0.1125341000000013
(416, 416, 3)
Found 0 boxes for img
0.11096620000000001
(416, 416, 3)
Found 0 boxes for img
0.12248200000000153
(416, 416, 3)
Found 0 boxes for img
0.11655689999999908
(416, 416, 3)
Found 0 boxes for img
0.11539580000000171
(416, 416, 3)
Found 0 boxes for img
0.12302379999999857
(416, 416, 3)
Found 0 boxes for img
0.1136070999999994
(416, 416, 3)
Found 0 boxes for img
0.1105678999999995
(416, 416, 3)
Found 1 boxes for img
raccoon 0.80 (467, 117) (1178, 622)
0.12929559999999896
(416, 416, 3)
Found 1 boxes for img
raccoon 0.86 (462, 83) (1182, 648)
0.12783220000000028
(416, 416, 3)
Found 1 boxes for img
raccoon 0.81 (498, 44) (1167, 699)
0.11988780000000077
(416, 416, 3)
Found 1 boxes for img
raccoon 0.81 (496, 47) (1170, 697)


Found 1 boxes for img
raccoon 0.80 (0, 23) (1018, 635)
0.11355299999999602
(416, 416, 3)
Found 1 boxes for img
raccoon 0.91 (0, 39) (1231, 627)
0.11296339999999816
(416, 416, 3)
Found 1 boxes for img
raccoon 0.89 (0, 35) (1237, 634)
0.11485659999999598
(416, 416, 3)
Found 1 boxes for img
raccoon 0.88 (0, 0) (1162, 686)
0.11303499999999644
(416, 416, 3)
Found 1 boxes for img
raccoon 0.93 (0, 12) (1233, 646)
0.11430510000000282
(416, 416, 3)
Found 1 boxes for img
raccoon 0.97 (0, 9) (1190, 663)
0.11282039999999682
(416, 416, 3)
Found 1 boxes for img
raccoon 0.92 (0, 9) (1203, 670)
0.11420220000000114
(416, 416, 3)
Found 1 boxes for img
raccoon 0.92 (0, 8) (1205, 670)
0.13209279999999524
(416, 416, 3)
Found 1 boxes for img
raccoon 0.82 (0, 0) (1149, 680)
0.11504159999999786
(416, 416, 3)
Found 1 boxes for img
raccoon 0.70 (0, 0) (1115, 677)
0.1128588999999991
(416, 416, 3)
Found 1 boxes for img
raccoon 0.73 (0, 0) (1110, 693)
0.11546780000000467
(416, 416, 3)
Found 1 boxes for img
raccoon

Found 0 boxes for img
0.11223920000000476
(416, 416, 3)
Found 0 boxes for img
0.11040020000000084
(416, 416, 3)
Found 0 boxes for img
0.1122307000000049
(416, 416, 3)
Found 0 boxes for img
0.11099389999999687
(416, 416, 3)
Found 0 boxes for img
0.11316039999999816
(416, 416, 3)
Found 0 boxes for img
0.11157110000000614
(416, 416, 3)
Found 0 boxes for img
0.11247269999999787
(416, 416, 3)
Found 0 boxes for img
0.11248210000000114
(416, 416, 3)
Found 1 boxes for img
kangaroo 0.50 (113, 121) (352, 432)
0.11426389999999742
(416, 416, 3)
Found 0 boxes for img
0.11193129999999485
(416, 416, 3)
Found 1 boxes for img
kangaroo 0.56 (111, 129) (356, 431)
0.11445489999999836
(416, 416, 3)
Found 1 boxes for img
kangaroo 0.64 (112, 145) (351, 430)
0.11338880000000273
(416, 416, 3)
Found 1 boxes for img
kangaroo 0.64 (117, 143) (347, 436)
0.1151185999999953
(416, 416, 3)
Found 1 boxes for img
kangaroo 0.60 (129, 146) (340, 445)
0.11345279999999747
(416, 416, 3)
Found 1 boxes for img
kangaroo 0.61 (1

Found 0 boxes for img
0.1266331000000065
(416, 416, 3)
Found 0 boxes for img
0.11096589999999651
(416, 416, 3)
Found 0 boxes for img
0.11231789999999364
(416, 416, 3)
Found 0 boxes for img
0.11127219999998772
(416, 416, 3)
Found 0 boxes for img
0.11242919999999401
(416, 416, 3)
Found 0 boxes for img
0.11091979999999069
(416, 416, 3)
Found 0 boxes for img
0.11245130000000358
(416, 416, 3)
Found 0 boxes for img
0.11087320000000034
(416, 416, 3)
Found 0 boxes for img
0.11213639999999714
(416, 416, 3)
Found 0 boxes for img
0.11121860000000083
(416, 416, 3)
Found 0 boxes for img
0.11179900000000487
(416, 416, 3)
Found 0 boxes for img
0.11042059999999765
(416, 416, 3)
Found 0 boxes for img
0.11225410000000124
(416, 416, 3)
Found 0 boxes for img
0.1106153000000063
(416, 416, 3)
Found 0 boxes for img
0.11228340000000969
(416, 416, 3)
Found 0 boxes for img
0.1118370000000084
(416, 416, 3)
Found 0 boxes for img
0.1125897000000009
(416, 416, 3)
Found 0 boxes for img
0.11033159999999498
(416, 416,

...

Found 2 boxes for img
raccoon 0.54 (39, 0) (1242, 695)
kangaroo 0.61 (202, 0) (1083, 720)
0.11659089999997718
(416, 416, 3)
Found 1 boxes for img
kangaroo 0.91 (203, 0) (1073, 720)
0.11911010000000033
(416, 416, 3)
Found 1 boxes for img
kangaroo 0.88 (191, 0) (1077, 720)
0.1292329000000052
(416, 416, 3)
Found 1 boxes for img
kangaroo 0.90 (180, 0) (1087, 715)
0.12852960000000735
(416, 416, 3)
Found 1 boxes for img
kangaroo 0.91 (185, 0) (1083, 713)
0.13861229999997704
(416, 416, 3)
Found 1 boxes for img
kangaroo 0.90 (184, 0) (1084, 700)
0.1273458000000005
(416, 416, 3)
Found 1 boxes for img
kangaroo 0.89 (175, 0) (1095, 720)
0.12366249999999468
(416, 416, 3)
Found 1 boxes for img
kangaroo 0.90 (153, 0) (1118, 708)
0.12325480000001221
(416, 416, 3)
Found 1 boxes for img
raccoon 0.54 (113, 0) (1146, 702)
0.12827750000002425
(416, 416, 3)
Found 1 boxes for img
raccoon 0.54 (113, 0) (1146, 702)
0.12630769999998392
(416, 416, 3)
Found 1 boxes for img
kangaroo 0.78 (104, 0) (1156, 662)
0.12

Found 0 boxes for img
0.11274130000001037
(416, 416, 3)
Found 0 boxes for img
0.11112689999998793
(416, 416, 3)
Found 1 boxes for img
kangaroo 0.84 (578, 330) (618, 551)
0.11529809999998974
(416, 416, 3)
Found 1 boxes for img
kangaroo 0.90 (575, 322) (620, 551)
0.12788380000000643
(416, 416, 3)
Found 2 boxes for img
kangaroo 0.62 (838, 285) (881, 495)
kangaroo 0.94 (570, 308) (623, 546)
0.11922099999998181
(416, 416, 3)
Found 2 boxes for img
kangaroo 0.62 (838, 285) (881, 495)
kangaroo 0.94 (570, 308) (623, 546)
0.11788549999999987
(416, 416, 3)
Found 1 boxes for img
kangaroo 0.92 (572, 316) (622, 539)
0.11392680000000155
(416, 416, 3)
Found 1 boxes for img
kangaroo 0.89 (573, 333) (622, 526)
0.11476070000000504
(416, 416, 3)
Found 2 boxes for img
kangaroo 0.62 (822, 335) (894, 507)
kangaroo 0.85 (576, 341) (622, 521)
0.11542289999999866
(416, 416, 3)
Found 0 boxes for img
0.11275750000001494
(416, 416, 3)
Found 0 boxes for img
0.11212299999999686
(416, 416, 3)
Found 0 boxes for img
0.

Found 1 boxes for img
raccoon 0.96 (0, 0) (1280, 720)
0.1138554000000056
(416, 416, 3)
Found 1 boxes for img
raccoon 0.96 (0, 0) (1280, 720)
0.11306220000000167
(416, 416, 3)
Found 1 boxes for img
raccoon 0.99 (0, 0) (1280, 720)
0.11413840000000164
(416, 416, 3)
Found 1 boxes for img
raccoon 0.99 (0, 0) (1280, 720)
0.11411730000000375
(416, 416, 3)
Found 1 boxes for img
raccoon 0.98 (0, 0) (1280, 720)
0.11427359999998998
(416, 416, 3)
Found 1 boxes for img
raccoon 0.99 (0, 0) (1280, 720)
0.11342440000001375
(416, 416, 3)
Found 1 boxes for img
raccoon 0.99 (0, 0) (1280, 720)
0.11368440000001101
(416, 416, 3)
Found 1 boxes for img
raccoon 0.96 (0, 0) (1280, 720)
0.11321159999999963
(416, 416, 3)
Found 1 boxes for img
raccoon 0.96 (0, 0) (1280, 720)
0.11370370000000207
(416, 416, 3)
Found 1 boxes for img
raccoon 0.99 (0, 0) (1280, 720)
0.11252319999999827
(416, 416, 3)
Found 1 boxes for img
raccoon 0.98 (0, 0) (1280, 720)
0.11372919999999453
(416, 416, 3)
Found 1 boxes for img
raccoon 0.9

Found 1 boxes for img
raccoon 0.92 (0, 0) (1280, 720)
0.1143403999999748
(416, 416, 3)
Found 1 boxes for img
raccoon 0.92 (0, 0) (1280, 720)
0.11464519999998402
(416, 416, 3)
Found 0 boxes for img
0.11408819999999764
(416, 416, 3)
Found 1 boxes for img
raccoon 0.52 (0, 0) (1280, 720)
0.11248209999999403
(416, 416, 3)
Found 1 boxes for img
raccoon 0.59 (0, 0) (1280, 720)
0.11596530000002758
(416, 416, 3)
Found 0 boxes for img
0.11147090000000048
(416, 416, 3)
Found 0 boxes for img
0.11270910000001777
(416, 416, 3)
Found 0 boxes for img
0.11185009999999806
(416, 416, 3)
Found 0 boxes for img
0.1118257999999912
(416, 416, 3)
Found 0 boxes for img
0.11127119999997603
(416, 416, 3)
Found 1 boxes for img
raccoon 0.65 (0, 0) (1280, 720)
0.12503760000001307
(416, 416, 3)
Found 1 boxes for img
raccoon 0.60 (0, 0) (1280, 720)
0.11483319999999253
(416, 416, 3)
Found 0 boxes for img
0.1113843999999915
(416, 416, 3)
Found 1 boxes for img
raccoon 0.68 (0, 0) (1280, 720)
0.11452779999999052
(416, 416

Found 0 boxes for img
0.11240159999999833
(416, 416, 3)
Found 0 boxes for img
0.11142609999998854
(416, 416, 3)
Found 0 boxes for img
0.1116888999999901
(416, 416, 3)
Found 0 boxes for img
0.11085740000001465
(416, 416, 3)
Found 1 boxes for img
raccoon 0.75 (19, 0) (1272, 720)
0.11443239999999832
(416, 416, 3)
Found 0 boxes for img
0.11079439999997476
(416, 416, 3)
Found 0 boxes for img
0.11476859999999078
(416, 416, 3)
Found 0 boxes for img
0.12453360000000657
(416, 416, 3)
Found 0 boxes for img
0.11196839999999497
(416, 416, 3)
Found 0 boxes for img
0.11151549999999588
(416, 416, 3)
Found 0 boxes for img
0.11283120000001645
(416, 416, 3)
Found 0 boxes for img
0.11170680000000743
(416, 416, 3)
Found 0 boxes for img
0.11155679999998824
(416, 416, 3)
Found 0 boxes for img
0.11125480000001176
(416, 416, 3)
Found 1 boxes for img
raccoon 0.57 (0, 0) (1280, 720)
0.11361440000001721
(416, 416, 3)
Found 0 boxes for img
0.11062970000000405
(416, 416, 3)
Found 1 boxes for img
raccoon 0.65 (17, 